In [ ]:
!nvidia-smi

Sun Dec 18 07:50:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip3 -q install yolov5==6.2.3

     |████████████████████████████████| 948 kB 6.9 MB/s 
     |████████████████████████████████| 88 kB 9.5 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 132 kB 71.5 MB/s 
     |████████████████████████████████| 79 kB 10.1 MB/s 
     |████████████████████████████████| 10.3 MB 47.9 MB/s 
     |████████████████████████████████| 140 kB 69.3 MB/s 
     |████████████████████████████████| 127 kB 61.0 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 3.2 MB 42.4 MB/s 
     |████████████████████████████████| 1.6 MB 44.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.0.4 which is incompatible.


In [ ]:
!rm -rf "train"
!rm -rf "test"
!rm -rf "sample_submission"

In [ ]:
!unzip "/content/drive/MyDrive/AV/DataverseHack22/input/train.zip" -d "/content" >> quit
!unzip "/content/drive/MyDrive/AV/DataverseHack22/input/sample_submission_nPrGNWH.zip" -d "/content" >> quit
!unzip "/content/drive/MyDrive/AV/DataverseHack22/input/test_niF99zC.zip" -d "/content" >> quit

In [ ]:
import os
import glob
import shutil
import numpy as np
import pandas as pd
from sklearn import model_selection
from tqdm import tqdm
from PIL import Image

import cv2
import matplotlib.pyplot as plt

In [ ]:
IMAGE_PATH = "/content/train/images"
OUTPUT_PATH = "/content/yolo_data"

os.makedirs(OUTPUT_PATH, exist_ok=True)

In [ ]:
def create_dataset(df, fold=0, data_type="train"):
    images_dir = f'{OUTPUT_PATH}/fold{fold}/images/{data_type}'
    labels_dir = f'{OUTPUT_PATH}/fold{fold}/labels/{data_type}'

    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for _, row in tqdm(df.iterrows(), total=len(df)):
        image_name = row['ImageID']
        name = image_name.split('.')[0]
        image = cv2.imread(os.path.join(IMAGE_PATH, image_name))
        # image = Image.open(os.path.join(IMAGE_PATH, image_name))
        H, W, _ = image.shape
        W = float(W)
        H = float(H)
        bounding_boxes = row['bboxes']
        yolo_data = []
        for bbox in bounding_boxes:

            xmin = bbox[0]
            xmax = bbox[1]
            ymin = bbox[2]
            ymax = bbox[3]

            x_center = (xmin + xmax) / 2
            y_center = (ymin + ymax) / 2
            w = xmax - xmin
            h = ymax - ymin

            x_center = x_center / W
            y_center = y_center / H
            w = w / W
            h = h / H

        
            yolo_data.append([0, x_center, y_center, w, h])
        yolo_data = np.array(yolo_data)
        np.savetxt(
            os.path.join(labels_dir, f"{name}.txt"),
            yolo_data,
            fmt=["%d", "%f", "%f", "%f", "%f"]
        )

        shutil.copyfile(
            os.path.join(IMAGE_PATH, image_name),
            os.path.join(images_dir, image_name)
        )

In [ ]:
df = pd.read_csv("/content/train/labels.csv")
df['bbox'] = df[['XMin', 'XMax', 'YMin', 'YMax']].values.tolist()
df = df.groupby("ImageID")['bbox'].apply(list).reset_index(name='bboxes')

kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(kfold.split(df)):

    df_train = df.iloc[train_idx].reset_index(drop=True)
    df_valid = df.iloc[valid_idx].reset_index(drop=True)

    print(df_train.shape, df_valid.shape)

    create_dataset(df_train, fold=fold, data_type="train")
    create_dataset(df_train, fold=fold, data_type="valid")

    yaml_file = f"""train: {OUTPUT_PATH}/fold{fold}/images/train
val: {OUTPUT_PATH}/fold{fold}/images/valid
nc: 1
names: ["pothole"]"""
    with open(f"{OUTPUT_PATH}/fold{fold}/data.yaml", 'w+') as f:
        f.write(yaml_file)

(539, 2) (135, 2)


100%|██████████| 539/539 [00:07<00:00, 75.48it/s]


(539, 2) (135, 2)


100%|██████████| 539/539 [00:04<00:00, 114.42it/s]


(539, 2) (135, 2)


100%|██████████| 539/539 [00:04<00:00, 110.83it/s]


(539, 2) (135, 2)


100%|██████████| 539/539 [00:04<00:00, 108.97it/s]


(540, 2) (134, 2)


100%|██████████| 540/540 [00:04<00:00, 110.36it/s]


In [ ]:
!python3 /usr/local/lib/python3.8/dist-packages/yolov5/train.py --project '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold0' \
                                                                --img 640 \
                                                                --batch-size 16 \
                                                                --epochs 50 \
                                                                --data /content/yolo_data/fold0/data.yaml \
                                                                --weights yolov5x6.pt \
                                                                --cache disk \
                                                                --seed 42 \
                                                                --optimizer SGD \
                                                                --patience 20 \
                                                                --workers 2 \
                                                                --noplots 

train: weights=yolov5x6.pt, cfg=, data=/content/yolo_data/fold0/data.yaml, hyp=../usr/local/lib/python3.8/dist-packages/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=True, evolve=None, bucket=, cache=disk, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold0, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=20, freeze=[0], save_period=-1, seed=42, local_rank=-1, mmdet_tags=False, entity=None, bbox_interval=-1, artifact_alias=latest, neptune_token=None, neptune_project=None, s3_upload_dir=None, upload_dataset=False
requirements: /usr/local/lib/python3.8/dist-packages/yolov5/requirements.txt not found, check failed.
YOLOv5 🚀 2022-12-18 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01

In [ ]:
!python3 /usr/local/lib/python3.8/dist-packages/yolov5/train.py --project '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold1' \
                                                                --img 640 \
                                                                --batch-size 16 \
                                                                --epochs 50 \
                                                                --data /content/yolo_data/fold1/data.yaml \
                                                                --weights yolov5x6.pt \
                                                                --cache disk \
                                                                --seed 42 \
                                                                --optimizer SGD \
                                                                --patience 20 \
                                                                --workers 2 \
                                                                --noplots 

train: weights=yolov5x6.pt, cfg=, data=/content/yolo_data/fold1/data.yaml, hyp=../usr/local/lib/python3.8/dist-packages/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=True, evolve=None, bucket=, cache=disk, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold1, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=20, freeze=[0], save_period=-1, seed=42, local_rank=-1, mmdet_tags=False, entity=None, bbox_interval=-1, artifact_alias=latest, neptune_token=None, neptune_project=None, s3_upload_dir=None, upload_dataset=False
requirements: /usr/local/lib/python3.8/dist-packages/yolov5/requirements.txt not found, check failed.
YOLOv5 🚀 2022-12-17 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01

In [ ]:
!python3 /usr/local/lib/python3.8/dist-packages/yolov5/train.py --project '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold2' \
                                                                --img 640 \
                                                                --batch-size 16 \
                                                                --epochs 50 \
                                                                --data /content/yolo_data/fold2/data.yaml \
                                                                --weights yolov5x6.pt \
                                                                --cache disk \
                                                                --seed 42 \
                                                                --optimizer SGD \
                                                                --patience 20 \
                                                                --workers 2 \
                                                                --noplots 

train: weights=yolov5x6.pt, cfg=, data=/content/yolo_data/fold2/data.yaml, hyp=../usr/local/lib/python3.8/dist-packages/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=True, evolve=None, bucket=, cache=disk, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold2, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=20, freeze=[0], save_period=-1, seed=42, local_rank=-1, mmdet_tags=False, entity=None, bbox_interval=-1, artifact_alias=latest, neptune_token=None, neptune_project=None, s3_upload_dir=None, upload_dataset=False
requirements: /usr/local/lib/python3.8/dist-packages/yolov5/requirements.txt not found, check failed.
YOLOv5 🚀 2022-12-17 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01

In [ ]:
!python3 /usr/local/lib/python3.8/dist-packages/yolov5/train.py --project '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold3' \
                                                                --img 640 \
                                                                --batch-size 16 \
                                                                --epochs 50 \
                                                                --data /content/yolo_data/fold3/data.yaml \
                                                                --weights yolov5x6.pt \
                                                                --cache disk \
                                                                --seed 42 \
                                                                --optimizer SGD \
                                                                --patience 20 \
                                                                --workers 2 \
                                                                --noplots 

train: weights=yolov5x6.pt, cfg=, data=/content/yolo_data/fold3/data.yaml, hyp=../usr/local/lib/python3.8/dist-packages/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=True, evolve=None, bucket=, cache=disk, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold3, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=20, freeze=[0], save_period=-1, seed=42, local_rank=-1, mmdet_tags=False, entity=None, bbox_interval=-1, artifact_alias=latest, neptune_token=None, neptune_project=None, s3_upload_dir=None, upload_dataset=False
requirements: /usr/local/lib/python3.8/dist-packages/yolov5/requirements.txt not found, check failed.
YOLOv5 🚀 2022-12-17 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01

In [ ]:
!python3 /usr/local/lib/python3.8/dist-packages/yolov5/train.py --project '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold4' \
                                                                --img 640 \
                                                                --batch-size 16 \
                                                                --epochs 50 \
                                                                --data /content/yolo_data/fold4/data.yaml \
                                                                --weights yolov5x6.pt \
                                                                --cache disk \
                                                                --seed 42 \
                                                                --optimizer SGD \
                                                                --patience 20 \
                                                                --workers 2 \
                                                                --noplots 

train: weights=yolov5x6.pt, cfg=, data=/content/yolo_data/fold4/data.yaml, hyp=../usr/local/lib/python3.8/dist-packages/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=True, evolve=None, bucket=, cache=disk, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=2, project=/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold4, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=20, freeze=[0], save_period=-1, seed=42, local_rank=-1, mmdet_tags=False, entity=None, bbox_interval=-1, artifact_alias=latest, neptune_token=None, neptune_project=None, s3_upload_dir=None, upload_dataset=False
requirements: /usr/local/lib/python3.8/dist-packages/yolov5/requirements.txt not found, check failed.
YOLOv5 🚀 2022-12-17 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01

In [ ]:
!python3 /usr/local/lib/python3.8/dist-packages/yolov5/detect.py --weights '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold0/exp2/weights/best.pt' '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold1/exp/weights/best.pt' '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold2/exp/weights/best.pt' '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold3/exp/weights/best.pt' '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold4/exp/weights/best.pt' \
                                                                 --source '/content/test/images' \
                                                                 --imgsz 640 \
                                                                 --save-txt \
                                                                 --save-conf \
                                                                 --conf-thres 0.01 \
                                                                 --iou-thres 0.4 \
                                                                 --max-det 50

detect: weights=['/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold0/exp2/weights/best.pt', '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold1/exp/weights/best.pt', '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold2/exp/weights/best.pt', '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold3/exp/weights/best.pt', '/content/drive/MyDrive/AV/DataverseHack22/output/yolov5x6/fold4/exp/weights/best.pt'], source=/content/test/images, data=../usr/local/lib/python3.8/dist-packages/yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.05, iou_thres=0.3, max_det=50, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 2022-12-18 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB

In [ ]:
def yolo_to_pascal_voc(x_center, y_center, w, h,  image_w, image_h):
    w = w * image_w
    h = h * image_h
    x1 = ((2 * x_center * image_w) - w)/2
    y1 = ((2 * y_center * image_h) - h)/2
    x2 = x1 + w
    y2 = y1 + h
    return [x1, x2, y1, y2]

In [ ]:
final_dicts = []

txt_files = glob.glob("/content/runs/detect/exp2/labels/*.txt")
for fname in txt_files:
    values = np.loadtxt(fname)
    shape = values.shape
    image_name = f"{fname.split('/')[-1].split('.')[0]}.jpg"

    image = cv2.imread(f"/content/test/images/{image_name}")
    h, w = image.shape[:2]

    if len(shape) == 1:
        boxes = values[1:5]
        score = values[5]
        label = values[0]

        boxes = yolo_to_pascal_voc(boxes[0], boxes[1], boxes[2], boxes[3], 640, 640)

        batch_dict = {}
        batch_dict['XMin'] = boxes[0]
        batch_dict['XMax'] = boxes[1]
        batch_dict['YMin'] = boxes[2]
        batch_dict['YMax'] = boxes[3]
        batch_dict['Conf'] = score

        batch_dict['ImageID'] = image_name

        batch_dict['LabelName'] = 'pothole'
        batch_dict['XMax'] = batch_dict['XMax']/640 * w
        batch_dict['XMin'] = batch_dict['XMin']/640 * w
        batch_dict['YMax'] = batch_dict['YMax']/640 * h
        batch_dict['YMin'] = batch_dict['YMin']/640 * h

        final_dicts.append(batch_dict)

    else:
        for i in range(len(values)):
            boxes = values[i][1:5]
            score = values[i][5]
            label = values[i][0]

            boxes = yolo_to_pascal_voc(boxes[0], boxes[1], boxes[2], boxes[3], 640, 640)

            batch_dict = {}
            batch_dict['XMin'] = boxes[0]
            batch_dict['XMax'] = boxes[1]
            batch_dict['YMin'] = boxes[2]
            batch_dict['YMax'] = boxes[3]
            batch_dict['Conf'] = score

            batch_dict['ImageID'] = image_name

            batch_dict['LabelName'] = 'pothole'
            batch_dict['XMax'] = batch_dict['XMax']/640 * w
            batch_dict['XMin'] = batch_dict['XMin']/640 * w
            batch_dict['YMax'] = batch_dict['YMax']/640 * h
            batch_dict['YMin'] = batch_dict['YMin']/640 * h

            final_dicts.append(batch_dict)

In [ ]:
final_df = pd.DataFrame(final_dicts)

In [ ]:
final_df.shape

(6495, 7)

In [ ]:
final_df['ImageID'].nunique()

556

In [ ]:
final_df.to_csv("yolov5_x6_640_c01_iou4_5fold_sub.csv", index=False)

In [ ]:
final_df.head()

,XMin,XMax,YMin,YMax,Conf,ImageID,LabelName
0,0.000000,450.000000,139.000050,445.999950,0.023534,1091.jpg,pothole
1,188.999775,408.999825,344.999925,449.999775,0.369187,1091.jpg,pothole
2,1.999800,261.999900,74.999925,231.999975,0.520602,1091.jpg,pothole
3,110.999925,242.999775,107.000100,225.000000,0.890380,1091.jpg,pothole
4,1.999931,127.999789,112.000116,216.000068,0.026029,817.jpg,pothole
